In [10]:
import sys
from pyhdf.SD import SD, SDC
import numpy as np
import matplotlib.pyplot as plt
import glob
from glob import glob
import os
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import csv
import h5py
import numpy as np
import json
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import functions as f

def download_ref(folder, filename, datestr, MOD_ID, urlprefix):
    username_file = open("/home/fun/profile/modis_username.txt", "r")
    password_file = open("/home/fun/profile/modis_password.txt", "r")
    username = username_file.readline()
    password = password_file.readline()

    url = f.generate_modis_url(datestr, MOD_ID, urlprefix)
    url = url + filename
    if len(filename) == 0:
        filename = 'refer_index_' + MOD_ID + '_' + datestr + '.html'
        print("HTML:", filename)
        
    print('downloading file ', url)
    
    r = requests.get(url, auth = (username, password))

    if r.status_code == 200:
        print('writing to', folder + filename)
        if os.path.isfile(folder + filename):
            return 
        with open(folder + filename, 'wb') as out:
            for bits in r.iter_content():
                out.write(bits)    
    
def download_ref_html(folder, MOD_ID, dt, urlprefix):
    profile = folder + 'profile.json'
    file = open(profile)
    info = json.load(file)

    start = datetime.datetime.strptime(info['start'], '%Y-%m-%d')
#         begin_date = start - timedelta(days= int(dt[0:len(dt)-1]))
#         final_date = end + timedelta(days= int(dt[0:len(dt)-1]))

    begin_date = start.replace(year=start.year - 1, month=start.month, day=2)
    it_id = 0
    it_date = begin_date 
    while (it_id < 12): # JH: downdload 12 month of reference images , one for each month
        datestr = it_date.strftime('%Y-%m-%d')
    
        url = f.generate_modis_url(datestr, MOD_ID, urlprefix)
        download_ref(folder, "", datestr, MOD_ID, urlprefix)
        
        if os.path.isfile(folder + 'refer_index_' + MOD_ID + '_' + datestr + '.html'):
            it_date = it_date + relativedelta(months=1)
            it_date = it_date.replace(year=it_date.year, month=it_date.month, day=1)  #JH: always start with first day of month and try
            it_id = it_id + 1
        else:
            it_date = it_date + timedelta(days=1)

def download_ref_wrapped(folder, MOD_ID, dt, urlprefix, tiles):
    globprof = folder + 'profile.json'
    print("PROFILE:", globprof)
    with open(globprof) as file:
        prof = json.load(file)
        area = prof['info']['acres_burned']
        end = datetime.datetime.strptime(prof['end'], '%Y-%m-%d')
        cutoff_date = end.replace(year=2021, month=2, day=1)
    if (int(area) < 3000 or end > cutoff_date):
        return
    
    download_ref_html(folder, MOD_ID, dt, urlprefix)
    f.download_hdf(folder, MOD_ID, urlprefix, tiles, 'refer_index_')
    
tiles = [{
    'lat_min': 40,
    'lat_max': 50,
    'name': 'h08v04'
}, 
{
    'lat_min': 30,
    'lat_max': 40,
    'name': 'h08v05'
}]


In [11]:
radius = 25
fillvalue = -3000
# print(sys.argv[1])
wildfires = glob('/home/fun/wildfire_data/Summit*')

for wildfire in wildfires:
    MOD_ref_files = glob(wildfire + '/r_*.npy')
    if(len(MOD_ref_files) != 12):
        download_ref_wrapped(wildfire + '/', 'MOD13Q1.006', '32d', 'https://e4ftl01.cr.usgs.gov/MOLT/', tiles)
        f.write_imgs(wildfire, radius, fillvalue, tiles, 'r_')
    else:
        print(wildfire, 'ref exist')

PROFILE: /home/fun/wildfire_data/Summit_Fire_2022-08-03/profile.json
ALL FILES: []
PROFILE: /home/fun/wildfire_data/Summit_Complex_2017-07-31/profile.json
HTML: refer_index_MOD13Q1.006_2016-07-02.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.07.02/
HTML: refer_index_MOD13Q1.006_2016-07-03.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.07.03/
HTML: refer_index_MOD13Q1.006_2016-07-04.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.07.04/
HTML: refer_index_MOD13Q1.006_2016-07-05.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.07.05/
HTML: refer_index_MOD13Q1.006_2016-07-06.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.07.06/
HTML: refer_index_MOD13Q1.006_2016-07-07.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.07.07/
HTML: refer_index_MOD13Q1.006_2016-07-08.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.07.08

HTML: refer_index_MOD13Q1.006_2016-11-15.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.11.15/
HTML: refer_index_MOD13Q1.006_2016-11-16.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.11.16/
writing to /home/fun/wildfire_data/Summit_Complex_2017-07-31/refer_index_MOD13Q1.006_2016-11-16.html
HTML: refer_index_MOD13Q1.006_2016-12-01.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.12.01/
HTML: refer_index_MOD13Q1.006_2016-12-02.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2016.12.02/
writing to /home/fun/wildfire_data/Summit_Complex_2017-07-31/refer_index_MOD13Q1.006_2016-12-02.html
HTML: refer_index_MOD13Q1.006_2017-01-01.html
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2017.01.01/
writing to /home/fun/wildfire_data/Summit_Complex_2017-07-31/refer_index_MOD13Q1.006_2017-01-01.html
HTML: refer_index_MOD13Q1.006_2017-02-01.html
downloading file  https://e4ftl01.cr.usgs.gov/

downloading  MOD13Q1.A2017001.h08v05.006.2017020220045.hdf to  /home/fun/wildfire_data/Summit_Complex_2017-07-31/
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2017.01.01/MOD13Q1.A2017001.h08v05.006.2017020220045.hdf
writing to /home/fun/wildfire_data/Summit_Complex_2017-07-31/MOD13Q1.A2017001.h08v05.006.2017020220045.hdf
parsing /home/fun/wildfire_data/Summit_Complex_2017-07-31/refer_index_MOD13Q1.006_2017-02-02.html
downloading  MOD13Q1.A2017033.h08v05.006.2017053061509.hdf to  /home/fun/wildfire_data/Summit_Complex_2017-07-31/
downloading file  https://e4ftl01.cr.usgs.gov/MOLT/MOD13Q1.006/2017.02.02/MOD13Q1.A2017033.h08v05.006.2017053061509.hdf
writing to /home/fun/wildfire_data/Summit_Complex_2017-07-31/MOD13Q1.A2017033.h08v05.006.2017053061509.hdf
parsing /home/fun/wildfire_data/Summit_Complex_2017-07-31/refer_index_MOD13Q1.006_2017-03-06.html
downloading  MOD13Q1.A2017065.h08v05.006.2017082122516.hdf to  /home/fun/wildfire_data/Summit_Complex_2017-07-31/
download